In [1]:
import os
import re
import numpy as np
import pandas as pd
import sys
from pathlib import Path
from datetime import date, datetime,timedelta
from calendar import monthrange
from IPython.display import display,HTML
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)

DATA_PROCESSING_DIR = Path("/Users/ylin/ValtaWork/Data and Reporting")
sys.path.append(str(DATA_PROCESSING_DIR))

from DataProcessing import *
data = import_data()
property=property_input()
from RevenueReportHelpers import *

In [2]:
startdate = pd.to_datetime('2024-01-01')
enddate = pd.to_datetime('2026-01-31')
osbr = data[data["Listing"].str.contains("Cottage")]
mask = (osbr["checkin_date"]>=startdate) & (osbr["checkin_date"]<=enddate) |\
(osbr["checkout_date"]>=startdate) & (osbr["checkout_date"]<=enddate)
osbr = osbr.loc[mask]


In [3]:
startdate = pd.to_datetime('2026-01-01')
enddate = pd.to_datetime('2026-02-28')
osbr = data[data["Listing"].str.contains("Cottage")]
mask = (osbr["checkin_date"]>=startdate) & (osbr["checkin_date"]<=enddate) |\
(osbr["checkout_date"]>=startdate) & (osbr["checkout_date"]<=enddate)
osbr = osbr.loc[mask]
checkins = osbr.assign(Date = osbr["checkin_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkin=("checkin_date",'count'))
checkouts = osbr.assign(Date = osbr["checkout_date"].astype(str).str[:10]).groupby("Date", as_index=False).agg(checkout=("checkout_date",'count'))
counts = checkins.merge(checkouts,on="Date",how="outer")
df = pd.DataFrame({
    "Date": pd.date_range(startdate, enddate)
})
counts["Date"] = pd.to_datetime(counts["Date"])
counts = df.merge(counts,on="Date",how="left")
counts["Weekday"] = counts["Date"].dt.day_name()
counts["Date"] = counts["Date"].astype(str).str[:10]
counts = counts[["Date", "Weekday", "checkin", "checkout"]]
property.loc[property["Listing"]=="Cottage 12",'BEDROOMS'] = 3
osbr_sum = (osbr[["Listing","checkout_date"]].merge(property[["Listing","BEDROOMS"]],on="Listing",how="left")
            .pivot_table(index="checkout_date", columns="BEDROOMS",aggfunc="size",fill_value=0).reset_index()
            .assign(Date=lambda x: x["checkout_date"].astype(str).str[:10]))

counts = counts.merge(osbr_sum, on="Date", how="left").drop(columns=["checkout_date"])
out ={
    "counts": counts,
    "bookings": osbr
}
with pd.ExcelWriter("/Users/ylin/Google Drive/My Drive/01- Compensation Calculation/Working/Brittany_booking_counts.xlsx", engine="openpyxl") as ew:
     for sheet_name, df in out.items():
        sheet = sheet_name[:31]
        df.to_excel(ew, sheet_name=sheet, index=False, na_rep="")

#counts.to_excel("/Users/ylin/Google Drive/My Drive/01- Compensation Calculation/Working/Brittany_booking_counts.xlsx", index=False,na_rep="")

In [4]:
ratings_path = "/Users/ylin/Google Drive/My Drive/** Properties ** -- Valta/0_Cohosting/1-Reviews/Guesty reviews from Tech team/20260216 guesty_reviews.xlsx"
ratings = pd.read_excel(ratings_path)
ratings["createdAt"] = pd.to_datetime(ratings["createdAt"], errors="coerce")
idx = (
    ratings["nickname"].str.contains("Cottage", na=False)
    & (ratings["createdAt"] > pd.to_datetime("2025-10-31"))
    & (ratings["createdAt"] < pd.to_datetime("2026-03-01")) 
    & (ratings["Overall"].isin([5,10]))
)
#ratings.loc[idx]

In [5]:
## count occupancy days per month
occupancy, daily = cal_occupancy(osbr)

In [6]:
occs = (occupancy.groupby("yearmonth",as_index=False)
         .agg(occdays=("occdays", "sum")))
occs

,yearmonth,occdays
0,2025-12,22
1,2026-01,107
2,2026-02,58
3,2026-03,2
